# Altair Practice Lab

In this assignment we will be crafting a series of visualizations using
Altair to get practice working with real data in this context.

Your responses should be within the functions given,
using appropriate helper functions to help with clarity
and reduce redundancy.

## Rubric

The criteria to receive an S on this assignment is a good-faith attempt at each portion.

A good-faith attempt should either:

- be fundamentally correct producing the expected output with minimal deviations,
- **OR** contain an explanation of what does not work and _details on what was tried_.

**To receive an N, at least half of the assignment should have good-faith attempts.**

Your charts do not need to match the examples exactly!  They are helpful to get a sense of what you're after, but focus on the problem description.

## Introducing the Dataset

In the data directory you'll find three files:

**legislators.csv** which consists of ~7400 records representing state legislators, it has the following fields:

- name
- given_name
- family_name
- party: As reported by the state.
- gender: Male / Female / Other\*
- jurisdiction: This field contains an identifier for the state or jurisdiction, see below for details.
- district: the name of the district represented
- type: upper | lower - The classification of the legislative. Most states have both, but DC and NE only have an upper chamber.

Note: Accurate data on gender is hard to come by in many states. There may be irregularities in this field. This is also why this field does not make further distinctions beyond Male/Female/Other.

**populations.csv** 

A CSV file with a row with each state's population.  

**actions.csv**

A CSV file with legislative actions for a few states across a range of years:

- description: action text as reported by state
- date: date of action recorded
- classification: not used for this assignment
- state: 2-letter state postal code (note: these are upper case, other files use lower case)
- session: an identifier representing which session the action comes from (not used)

In [ ]:
import polars as pl
import pandas as pd
import altair as alt
import numpy as np
import datetime as dt
#import vegafusion
#import vegafusion_embed


## Part 1: Data Exploration

First, we'll build a few exploratory visualizations to get a sense of the data for this assignment.

### 1.0: Cleaning

As mentioned above, there is no 'state' field. Jurisdiction is in the format:

`ocd-jurisdiction/country:us/state:nc/government`

and for non-states:

- `ocd-jurisdiction/country:us/district:dc/government`
- `ocd-jurisdiction/country:us/territory:pr/government`

So for our purposes, we want to add a `state` column from the two letter code after either "state:", "district:", or "territory:".
(We will treat DC and PR as states.)

Complete the function `legislators_df` which should return the data from `legislators.csv` in a dataframe, with an additional `state` column.

In [ ]:
def juris_split(cell):
    """
    This function takes a cell in the jurisdiction column of legislator.csv and 
    returns the name of the state/territory referenced in that cell.
    Takes:
        cell (str): A string from the jurisdiction column such as:
            ocd-jurisdiction/country:us/district:dc/government
    Returns
        state (str): The state referenced in that string.
    """
    j_list = cell.split("/")
    state = j_list[2].split(":")[1]
    return state

def legislators_df():
    """
    Takes:
        Nothing, but it looks for a csv file at the path data/legislators.csv
    Returns:
        leg_df a polars dataframe with the state column appended
    """
    leg_df = pl.read_csv("data/legislators.csv")
    # To understand how to vectorize a function in polars I asked Perplexity AI the following:
    #       How do I apply a vectorized function in polars?
    leg_df = leg_df.with_columns(leg_df["jurisdiction"].map_elements(juris_split, pl.String).alias("state"))
    return leg_df

# render dataframe
legislators_df()

### 1.1: Initial Plot

First let's build a visualization of gender breakdowns in state legislatures.
Use the following:

- stacked bars per state
- each segment of stacked bar is gender

Your graph should somewhat resemble *imgs/ex1.1.png*.

In [ ]:
def states_by_gender_initial(df):
    grouped_df = df.group_by("state","gender").agg(pl.col("name").count())

    chart = alt.Chart(grouped_df).mark_bar().encode(
        x = alt.X('state:N').title("State"),
        y = alt.Y('name:Q').title("Count of Legislators"),
        color = 'gender:N')
    return chart

# render chart
states_by_gender_initial(legislators_df())

### 1.2: Improvements

While it is clear from the first chart that there are more elected officials that are men than women, it is hard to compare across states.

Make the following adjustments:

- Normalize the chart so that each bar is a percentage, allowing for direct comparison across states.
- Since this is US political data, the colors red and blue have a strong meaning, associated with the Republican and Democratic parties. Change the color scheme to avoid red and blue. (I chose #8624f5 for women and #1fc3aa for men based on this article: <https://blog.datawrapper.de/gendercolor/>)
- Two states are very close to 50%, add a line at 50% using a layered chart to make it easier to see if they exceed 50% or not.

Your graph should somewhat resemble *imgs/ex1.2.png*.

In [ ]:
def states_by_gender_improved(df):
    grouped_df = df.group_by("state","gender").agg(pl.col("name").count().alias("total")).with_columns(
            [pl.col('total').sum().over('state'),
             (pl.col("total")/pl.col("total").sum().over("state") * 100).alias("percentage")])
    #Getting the grouping was tricky, I asked Perplexity AI the following:
        #I am using polars to aggregate my data into two groups. 
        #How can I calculate the percentage of one group within another group?
    chart = alt.Chart(grouped_df).mark_bar().encode(
    x = alt.X('state:N').title("State"),
    y = alt.Y('percentage:Q').title("Percentage of Legislators"),
    color = alt.Color('gender:N', scale = alt.Scale(
        domain = ["Female", "Male", "Other"],
        range = ["fuchsia", "lawngreen", "wheat"]
    )))
    # This source helped work out how to set color:
      #  https://www.geeksforgeeks.org/how-to-color-a-scatter-plot-by-a-variable-in-altair/
    line = alt.Chart(pd.DataFrame({'y': [50]})).mark_rule().encode(y='y')
    return chart + line

# render chart
states_by_gender_improved(legislators_df())

## Part 2: Party Breakdown

We'll now take a look at party control. We can start with essentially the same chart.

### 2.0 - Party Control

Copy your code from 1.2 above & modify it to use party instead of gender.  Your graph will wind up with too many parties, see `imgs/ex2.0.png`.

In [ ]:
def party_control_raw(df):
    grouped_df = df.group_by("state","party").agg(pl.col("name").count().alias("total")).with_columns(
            [pl.col('total').sum().over('state'),
             (pl.col("total")/pl.col("total").sum().over("state") * 100).alias("percentage")])

    chart = alt.Chart(grouped_df).mark_bar().encode(
    x = alt.X('state:N').title("State"),
    y = alt.Y('percentage:Q').title("Percentage of Legislators"),
    color ='party:N'
    )
    line = alt.Chart(pd.DataFrame({'y': [50]})).mark_rule().encode(y='y')
    return chart + line
    return chart

party_control_raw(legislators_df())

### 2.1 - Cleaning Data

The above graph still has some shortcomings:

- Most states have an upper and lower chamber, and party control may vary between them. We'll need to make two bars per state (which we'll tackle in 2.2).
- Also, there are too many variations of party as you can see here:

Let's transform the data again, adding a new column "party_code" with the following rules:

- if the word 'Democratic' appears, set party_code to 'D'
- if the word 'Republican' appears, set the party_code to 'R'
- otherwise, set the party_code to 'O'

Party data in NE, DC, and PR does not work with this scheme.
For simplicity, we will exclude them from our analysis.

For this portion, implement `clean_party_df` which should return a modified legislators DataFrame with the `party_code` column, and the rows for states 'DC', 'NE' and 'PR' dropped.

In [ ]:
def party_code(party):
    if "Democratic" in party:
        return "D"
    elif "Republican" in party:
        return 'R'
    else:
        return 'O'
 
def clean_party_df():
    leg_df = legislators_df()
    leg_df = leg_df.with_columns(leg_df["party"].map_elements(party_code, pl.String).alias("party_code"))
    return leg_df

clean_party_df()

### 2.2 - Faceted Plot

Add a function `party_control_by_chamber` that contains the following elements:

- One bar per state, **along the Y axis**.
- Each bar should consist of a stack: a blue portion, a green portion, and a red portion, corresponding to the D, O, and R `party_code` respectively.
- A vertical line at the 50% mark, indicating (likely) party control.
- Finally, facet the chart on `type` so that you get a set of bars for the lower and upper chambers.

See `imgs/ex2.2.png` for an example.

In [ ]:
def party_control_by_chamber(df):
    
    #This post helped me understand how facet wrap works with a line once I started getting errors:
    # https://github.com/vega/altair/issues/1124 
    grouped_df = df.group_by("state","type", "party_code").agg(pl.col("name").count().alias("total")).with_columns(
        [pl.col('total').sum().over('state'),
         (pl.col("total")/pl.col("total").sum().over("state") * 100).alias("percentage")])
    # I used this link for assistance with Facet Wrap:
       # https://www.geeksforgeeks.org/understanding-facet-wrap-in-altair/?ref=next_article
    chart = alt.Chart().mark_bar().encode(
        x = alt.Y('percentage:Q').title("Percentage of Legislators"),
        y = alt.X('state:N').title("State"),
        color = alt.Color('party_code:N', scale = alt.Scale(
        domain = ["D", "O", "R"],
        range = ["blue", "green", "red"])),
    )
    line = alt.Chart(pd.DataFrame({'x': [50]})).mark_rule().encode(x='x')
    final = (chart + line).facet(column = "type:N", data = grouped_df)
    return final

party_control_by_chamber(clean_party_df())

## Part 3: Comparing by Population

For part three, we are interested in the relationship of various properties of legislatures to the total population of the state.

To do this, we'll need to create a combined DataFrame that mixes in data from `populations.csv`.

### 3.0 - Create Combined DataFrame

Write the function `population_combined_df`, which should return a DataFrame with the columns:

- state: abbreviation of state
- upper: total seats in upper chamber
- lower: total seats in lower chamber
- pop_2020: the 2020 population, obtained from merging with `population.csv`

**Data Note:** These numbers are based on the non-vacant seats as-of a particular day in September 2024. Vacancies will cause the counts to be off by a bit, but the general size of the legislature should be roughly the same.

In [ ]:
def population_combined_df():
    leg_df = legislators_df()
    leg_df = leg_df.group_by("state", "type").agg(pl.col("name").count().alias("total")).with_columns(
        pl.col('total').sum().over('state', 'type'))
    leg_df = leg_df.filter((pl.col("type") == "upper") | (pl.col("type") == "lower"))
    leg_df = leg_df.pivot("type", index="state", values = "total")
    pop_df = pl.read_csv("data/populations.csv")
    full_df = leg_df.join(pop_df, "state", "inner")
    
    return full_df

population_combined_df()

### 3.1 - Create Population vs. Seats Scatterplot

Create a new plot with two layers:

- Population on the X axis
- Number of seats on the Y axis
- Upper chamber points should be purple and use the 'triangle-up' shape.
- Lower chamber points should be orange and use the 'triangle-down' shape.
- Make a customization or two to your chart's default labels and axes, whatever you feel is appropriate.

Hint: You can layer two charts for this.

In [ ]:
def scatter_pop_size():
    df = population_combined_df()

    chart_upper = alt.Chart(df).mark_point(size=60).encode(
        x = alt.X("pop_2020:Q"),
        y = alt.Y('upper:Q'),
        color=alt.value("purple"),
        shape = alt.value('triangle-up'))

    chart_lower = alt.Chart(df).mark_point(size=60).encode(
        x = alt.X("pop_2020:Q").title("2020 Population"),
        y = alt.Y('lower:Q').title("Number of Legislative Seats"),
        color=alt.value("orange"),
        shape = alt.value('triangle-down'))
    
    chart = (chart_upper + chart_lower).properties(
        title = "Seats in Legislature by State Population")
    return chart

def scatter_pop_size_old():
    df = population_combined_df()
    colors = ["purple", "orange"]
    shapes = ['triangle-up', 'triangle-down']
    chart = alt.Chart(df).mark_point(size=60).encode(
        x = alt.X("pop_2020:Q").title("2020 Population"),
        y = alt.Y(alt.repeat('layer')).title("Number of Seats"),
        color=alt.ColorDatum(alt.repeat('layer')),
        shape = alt.ShapeDatum(alt.repeat('layer'))).repeat(
        layer=["upper", "lower"]).configure_range(
        category=alt.RangeScheme(colors),
        symbol = alt.RangeScheme(shapes))
    # This post helped me understand how to use repeat and color/shape things differently:
       # https://stackoverflow.com/questions/72488938/setting-colors-in-altair-layered-chart 
    return chart
scatter_pop_size()

### 3.2 - Regressions

Add two more layers, a purple & orange regression line for each chamber.  See `imgs/ex3.2.png`

Hint: See `transform_regression`.

In [ ]:
def scatter_pop_size_regression():
    df = population_combined_df()
    chart_upper = alt.Chart(df).mark_point(size=60).encode(
        x = alt.X("pop_2020:Q"),
        y = alt.Y('upper:Q'),
        color=alt.value("purple"),
        shape = alt.value('triangle-up'))

    chart_upper = chart_upper + chart_upper.transform_regression('pop_2020', 'upper').mark_line()

    chart_lower = alt.Chart(df).mark_point(size=60).encode(
        x = alt.X("pop_2020:Q").title("2020 Population"),
        y = alt.Y('lower:Q').title("Number of Legislative Seats"),
        color=alt.value("orange"),
        shape = alt.value('triangle-down'))

    chart_lower = chart_lower + chart_lower.transform_regression('pop_2020', 'lower').mark_line()
    
    chart = (chart_upper + chart_lower).properties(
        title = "Seats in Legislature by State Population")

    return chart

scatter_pop_size_regression()

## Part 4: Actions Heatmap

The file `actions_il-in-mi-wi_2021-2024.csv` contains nearly half a million records, representing every official action taken on a piece of legislation in these four midwestern states over the past two sessions.

Legislatures work quite differently, some meet all year, while others meet for very short periods.
By creating a heatmap of what days actions take place, we can get a sense of how different states compare.

### 4.0 - Load Actions

Complete `actions_df`, which should load the data from `actions_il-in-mi-wi_2021-2024.csv`.

Tips: 
- Make sure that the `date` column is loaded as a date type!
- Dates are in YYYY-MM-DD format, though some also have additional characters for time, which you will want to ignore.

In [ ]:
def date_conv(cell):
    """
    This function chops off strings at the "T" character so that they can uniformly
    be converted to dates.
    Takes:
        cell (str): A date cell from the actions data frame.
    Returns:
        date (str): A string that can easily be formatted into a date.
    """
    date = cell
    if "T" in cell:
        list = cell.split("T")
        date = list[0]
    return date

def actions_df():
    df = pl.read_csv("data/actions_il-in-mi-wi_2021-2024.csv")
    df = df.with_columns(pl.col("date").map_elements(date_conv, pl.String))
    df = df.with_columns(pl.col("date").str.strptime(pl.Date))
    return df
    
actions_df()    

### 4.1 - Actions Heatmap

Generate a heatmap (using `mark_rect`) with:

- a row per state
- each row consists of shaded marks with shading based on the total action count for a given week

Tip: Use the 'yearweek(date)' aggregation for the X channel.

See `imgs/ex4.1.png`.

In [ ]:
# We need to disable max rows to do the aggregation in Altair
alt.data_transformers.disable_max_rows()
def actions_heatmap_scaled(df):
    df_actions = actions_df()
    chart = alt.Chart(df).mark_rect().encode(
        y='state:N',
        x= 'yearweek(date)')
    return chart

actions_heatmap_scaled(actions_df())

### 4.2 - Excluding IL Outliers

Illinois clearly dominates the above graph, below modify two calls to `actions_heatmap` with a modified dataframe that excludes IL, and a modified dataframe that only includes IL.

(Note how by using functions in our dataframe we can more easily reuse portions by making small adjustments to the data.)

See `ex4.2a.png` and `ex4.2b.png`

In [ ]:
il_only = actions_df()
il_only = il_only.filter(pl.col("state") == "IL")
actions_heatmap_scaled(il_only)

In [ ]:
sans_il = actions_df()
sans_il = sans_il.filter(pl.col("state") !="IL")
actions_heatmap_scaled(sans_il)

#### 4.3 - Cumulative Line Chart

Another way to view this data would be with a cumulative line chart.

Create a chart with:

- days on the X axis
- cumulative actions to date on the Y axis
- one line per state

Hint: To do this you will need to look at the `transform_window` function.

See `ex4.3.png` for an example.

In [ ]:
def actions_cumulative(df):
    df_actions = df
    chart = alt.Chart(df_actions, width=600).mark_line().transform_window(
        sort=[{'field': 'date'}],
        frame=[None, 0],
        cumulative_actions = 'count(date)'
    ).encode(
        x = 'date:O',
        y = 'cumulative_wheat:Q')
    return chart
        
actions_cumulative(actions_df())